<a href="https://colab.research.google.com/github/mosmtilk222/Data-Analysis-Projects/blob/main/my_aoe_matches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests

In [ ]:
import requests
import math
import json
import sqlite3
import pandas as pd
import bokeh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

def run_query(query, parameters = ()):
      with sqlite3.connect('/content/drive/My Drive/Colab Notebooks/AOE/database.db') as conn:
        cursor = conn.cursor()
        result = cursor.execute(query, parameters)
        conn.commit()
      return 

Mounted at /content/drive


In [ ]:
civilizations = {1: 'Britons',
 2: 'Franks',
 3: 'Goths',
 4: 'Teutons',
 5: 'Japanese',
 6: 'Chinese',
 7: 'Byzantines',
 8: 'Persians',
 9: 'Saracens',
 10: 'Turks',
 11: 'Vikings',
 12: 'Mongols',
 13: 'Celts',
 14: 'Spanish',
 15: 'Aztecs',
 16: 'Mayans',
 17: 'Huns',
 18: 'Koreans',
 19: 'Italians',
 20: 'Indians',
 21: 'Incas',
 22: 'Magyars',
 23: 'Slavs',
 24: 'Portuguese',
 25: 'Ethiopians',
 26: 'Malians',
 27: 'Berbers',
 28: 'Khmer',
 29: 'Malay',
 30: 'Burmese',
 31: 'Vietnamese',
 32: 'Bulgarians',
 33: 'Tatars',
 34: 'Cumans',
 35: 'Lithuanians',
 36: 'Burgundians',
 37: 'Sicilians',
 38: 'Poles',
 39: 'Bohemians'}

# **Extract**

Getting my latest 1 v 1 Random Map matches in the next 3 blocks and adding them to the database

In [ ]:
my_steam_id = 76561198151358712
response = requests.get(f'https://aoe2.net/api/player/matches?game=aoe2de&steam_id={my_steam_id}&count=1000')
matches = response.json()

In [ ]:
def match_data(json):
  if json["num_players"] == 2 and json["leaderboard_id"] == 3:
    num_civ_p1 = json["players"][0]["civ"]
    civ_p1 = civilizations[num_civ_p1]
    nick_p1 = json["players"][0]["name"]
    elo_p1 = json["players"][0]["rating"]

    num_civ_p2 = json["players"][1]["civ"]
    civ_p2 = civilizations[num_civ_p2]
    nick_p2 = json["players"][1]["name"]
    elo_p2 = json["players"][1]["rating"]

    time = json["finished"] - json["started"]
    time = math.trunc(time / 60)

    if nick_p1 == 'SuN | MostMilk':
      player = nick_p2
      my_civ = civ_p1
      enemy_civ = civ_p2
      enemy_elo = elo_p2
      my_elo = elo_p1
      victory = json["players"][0]["won"]
      
    else:
      player = nick_p1
      my_civ = civ_p2
      enemy_civ = civ_p1
      enemy_elo = elo_p1
      my_elo = elo_p2
      victory = json["players"][1]["won"]

    run_query('INSERT INTO Matches VALUES (?,?,?,?,?,?,?)',(player, my_civ, enemy_civ, enemy_elo, my_elo, time, victory))

In [ ]:
for match in matches:
  match_data(match)

## **Transform**

In [ ]:
conn = sqlite3.connect('/content/drive/My Drive/Colab Notebooks/AOE/database.db')

df = pd.read_sql('SELECT * FROM Matches', conn)
df

,Player,My civilization,Enemy civilization,Enemy elo,My elo,Time (minutes),Win
0,[GT]YellowJacket,Chinese,Byzantines,1855.0,1822.0,33,1.0
1,Combi | IamTeTas,Portuguese,Goths,1893.0,1810.0,2,1.0
2,Alfred,Chinese,Cumans,1720.0,1822.0,26,1.0
3,[Cotcot] Lighty,Huns,Vietnamese,1892.0,1822.0,29,0.0
4,AAOE | ManuelAntonio,Teutons,Goths,1730.0,1822.0,14,1.0
...,...,...,...,...,...,...,...
1019,[mPzzA] rainManzD,Vikings,Lithuanians,1801.0,1872.0,21,1.0
1020,Elite400,Celts,Chinese,1828.0,1872.0,25,1.0
1021,RediRodion,Italians,Japanese,1913.0,1872.0,9,0.0
1022,Spearsky,Goths,Lithuanians,1888.0,1872.0,29,1.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024 entries, 0 to 1023
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Player              1024 non-null   object 
 1   My civilization     1024 non-null   object 
 2   Enemy civilization  1024 non-null   object 
 3   Enemy elo           1011 non-null   float64
 4   My elo              1015 non-null   float64
 5   Time (minutes)      1024 non-null   int64  
 6   Win                 1022 non-null   float64
dtypes: float64(3), int64(1), object(3)
memory usage: 56.1+ KB


In [ ]:
df.loc[[10],["Player"]]

,Player
10,Cachinero
